# Проект "Предсказание вероятности подключения услуги"

## Постановка задачи

#### Задача

Требуется на основании имеющихся данных об абонентах Мегафон построить  алгоритм, который для каждой пары пользователь-услуга определит вероятность подключения услуги.

#### Целевая переменная

**target** - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно. 

#### Метрика качества

F1-score (sklearn.metrics.f1_score)

#### Требования к решению

*Целевая метрика*

* Метрика оценивается невзвешенным образом, , как например делает функция sklearn.metrics.f1_score(…, average=’macro’)

*Решение должно содержать*

1.	Работающая модель в формате pickle, которая принимает файл data_test.csv из корневой папки и записывает в эту же папку файл answers_test.csv. В этом файле должны находится 4 столбца: buy_time, id, vas_id и target. Target можно записать как вероятность подключения услуги.
2.	Тетрадка Jupyter Notebook с кодом модели. 
3.	Презентация в формате .pdf, в которой необходимо отразить:

   * Информация о модели, ее параметрах, особенностях и основных результатах.
   * Обоснование выбора модели и ее сравнение с альтернативами.
   * Принцип составления индивидуальных предложений для выбранных абонентов.

   Рекомендуемое количество слайдов – 5 – 10.

Файл answers_test.csv с результатами работы модели, презентацию, ноутбуки и резюме необходимо прикрепить ко второму уроку “курсовой проект”.

## Решение

### Обзор данных

В качестве исходных данных представлена информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить.

Отдельным набором данных является нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени.

Данные train и test разбиты по периодам – на train доступно 4 месяцев, а на test отложен последующий месяц. 

Итого, в качестве входных данных представлены:
* **data_train.csv**: 
    * id, 
    * vas_id, 
    * buy_time, 
    * target
* **features.csv.zip**: 
    * id, 
    * ..feature_list..

И тестовый набор:
* **data_test.csv**: 
    * id, 
    * vas_id, 
    * buy_time

**Описание датасета**
* **id** - идентификатор абонента
* **vas_id** - подключаемая услуга
* **buy_time** - время покупки, представлено в формате timestamp, для работы с этим столбцом понадобится функция datetime.fromtimestamp из модуля datetime.
* **target** - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно.

#### Подключение библиотек и скриптов

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import dask.dataframe as dd
from dask.distributed import Client, progress

#### Загрузка данных

Загрузим исходные .csv-файлы в dask.dataframe и посмотрим состав столбцов.

In [2]:
features_dask = dd.read_csv('features.csv',sep='\t')
train_dask = dd.read_csv('data_train.csv')
test_dask = dd.read_csv('data_test.csv')

In [3]:
train_dask.head()

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0
3,3,3535012,5.0,1535922000,0.0
4,4,1693214,1.0,1535922000,0.0


In [4]:
test_dask.head()

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400
3,3,1343255,5.0,1547413200
4,4,1277040,2.0,1546808400


In [5]:
features_dask.head()

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,-65.076097,-6.783660,-30.006538,-2.736081,-4.007526,-2.558912,49.520873,38.19189,-0.000725,-0.016435,-0.107041,-1.17746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-42.026959,-2841.496068,-1085.821501,-1755.674564,-89.504287,-119.724355,-70.712019,-54.069191,-16.642826,-7.896282,-5.634035,-10.717958,-28.571103,57.869716,52.911014,26.828289,26.668705,-4.958702,38.254749,-1.671324,-0.001656,1.318354,2.117335,-0.265234,0.331838,0.078356,-0.237576,0.254338,-0.028454,-0.044465,3.698872,26.411990,-0.036834,3.869969,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-3.447583,-0.040043,-9.408469,4.027863,-11.955314,-1.019293,-0.473446,-2.62084,-1087.017387,-1757.811263,-0.36799,0.396143,-2844.828651,-2298.725139,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,-0.058077,0.129549,0.274871,-3.618164,-11.681641,-0.573283,0.531557,0.582717,-190.670372,1.856777,3.277409,2.174027,4.064012,0.0,-1.276187,-0.020137,-0.042636,-29.797016,-70.470802,-14.96363,-34.888325,-3.861461,-0.317164,-0.007024,-0.143269,60.582329,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,-0.152157,-0.002595,-4.678214,-0.014542,0.280492,-21.183166,-44.376426,-25.320085,-51.984826,-23.961228,-54.128903,-11.614497,-30.288386,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,-1.257363,-2.793637,-1.932758,-5.008096,-9.978121,-16.684052,4.645192,13.112964,-0.034569,-0.163184,-109.036398,0.533317,-1.929048,0.376263,-0.228106,-0.251959,-0.000567,0.566264,-0.000708,-0.02921,0.895335,-0.001358,0.0,0.039208,0.665644,-0.008999,-11953.712824,-45175.257711,0.377099,-30.716053,-61790.157098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,0.377104,-0.00004,0.379228,-0.012257,-0.107878,9.595373e+08,-42.014078,-4.405604e+08,1.356414e+09,5.565998,-1.465191,-33.302382,-249.128986,-36.772492,-0.364694,-0.133771,-0.209468,-32.356505,-109.884564,-876.69102,-5.368281,-247.110707,-108.409742,-512.437331,-84.617978,-17.295406,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,-53.216097,-6.783660,-26.544905,-2.736081,-4.007526,-2.558912,67.300873,55.97189,-0.000725,-0.016435,-0.107041,15.77254,-3.178521,411.379185,-10.744164,-0.094251,-0.001733,-0.009327,131.407791,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-192.026959,-2937.657200,-1181.982633,-1755.674564,447.193953,1258.981645,-119.662019,-54.6025

В датасете features.csv имеется столбец buy_time - столбец с аналогичны именем имеется в датасетах data_train.csv, data_test.csv

Загрузим исходные .csv-файлы в dask.dataframe c одновременным переименованием столбцов:
* индексов
* buy_time

In [6]:
def mapper(name):
    return {'Unnamed: 0': f'index_{name}', 
            'buy_time': f'buy_time_{name}'}


features_df = dd.read_csv('features.csv',sep='\t').rename(columns=mapper('feat'))
train_df = dd.read_csv('./data/data_train.csv').rename(columns=mapper('vas'))
test_df = dd.read_csv('./data/data_test.csv').rename(columns=mapper('vas'))

In [7]:
train_df.head()

,index_vas,id,vas_id,buy_time_vas,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0
3,3,3535012,5.0,1535922000,0.0
4,4,1693214,1.0,1535922000,0.0


In [8]:
test_df.head()

,index_vas,id,vas_id,buy_time_vas
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400
3,3,1343255,5.0,1547413200
4,4,1277040,2.0,1546808400


In [9]:
features_df.head()

,index_feat,id,buy_time_feat,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,-65.076097,-6.783660,-30.006538,-2.736081,-4.007526,-2.558912,49.520873,38.19189,-0.000725,-0.016435,-0.107041,-1.17746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-42.026959,-2841.496068,-1085.821501,-1755.674564,-89.504287,-119.724355,-70.712019,-54.069191,-16.642826,-7.896282,-5.634035,-10.717958,-28.571103,57.869716,52.911014,26.828289,26.668705,-4.958702,38.254749,-1.671324,-0.001656,1.318354,2.117335,-0.265234,0.331838,0.078356,-0.237576,0.254338,-0.028454,-0.044465,3.698872,26.411990,-0.036834,3.869969,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-3.447583,-0.040043,-9.408469,4.027863,-11.955314,-1.019293,-0.473446,-2.62084,-1087.017387,-1757.811263,-0.36799,0.396143,-2844.828651,-2298.725139,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,-0.058077,0.129549,0.274871,-3.618164,-11.681641,-0.573283,0.531557,0.582717,-190.670372,1.856777,3.277409,2.174027,4.064012,0.0,-1.276187,-0.020137,-0.042636,-29.797016,-70.470802,-14.96363,-34.888325,-3.861461,-0.317164,-0.007024,-0.143269,60.582329,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,-0.152157,-0.002595,-4.678214,-0.014542,0.280492,-21.183166,-44.376426,-25.320085,-51.984826,-23.961228,-54.128903,-11.614497,-30.288386,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,-1.257363,-2.793637,-1.932758,-5.008096,-9.978121,-16.684052,4.645192,13.112964,-0.034569,-0.163184,-109.036398,0.533317,-1.929048,0.376263,-0.228106,-0.251959,-0.000567,0.566264,-0.000708,-0.02921,0.895335,-0.001358,0.0,0.039208,0.665644,-0.008999,-11953.712824,-45175.257711,0.377099,-30.716053,-61790.157098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,0.377104,-0.00004,0.379228,-0.012257,-0.107878,9.595373e+08,-42.014078,-4.405604e+08,1.356414e+09,5.565998,-1.465191,-33.302382,-249.128986,-36.772492,-0.364694,-0.133771,-0.209468,-32.356505,-109.884564,-876.69102,-5.368281,-247.110707,-108.409742,-512.437331,-84.617978,-17.295406,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,-53.216097,-6.783660,-26.544905,-2.736081,-4.007526,-2.558912,67.300873,55.97189,-0.000725,-0.016435,-0.107041,15.77254,-3.178521,411.379185,-10.744164,-0.094251,-0.001733,-0.009327,131.407791,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-192.026959,-2937.657200,-1181.982633,-1755.674564,447.193953,1258.981645,-119.662019,-54

In [10]:
LEN_TRAIN = len(train_df.index)
print(f'Наблюдений в "data_train.csv": {LEN_TRAIN}')

Наблюдений в "data_train.csv": 831653


In [11]:
LEN_TEST = len(test_df.index)
print(f'Наблюдений в "data_test.csv": {LEN_TEST}')

Наблюдений в "data_test.csv": 71231


#### Слияние тренировочной выборки и набора данных с профилем абонента

Предположим, что в features.csv имеются данные для всех id, которые имеются в тренировочной и тестовой выборках.

Проведем слияние методом пересечения (how='inner') по столбцу id.

In [12]:
train_merge = dd.merge(train_df, features_df, on=['id'], how='inner')

In [13]:
train_feat = train_merge.compute()

In [14]:
train_feat.head()

,index_vas,id,vas_id,buy_time_vas,target,index_feat,buy_time_feat,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
0,140,4130548,2.0,1544389200,0.0,8832,1548018000,11.700029,17.790888,4.429214,-10.646798,-0.988246,-31.75179,7.85382,29.933903,-3.863189,-14.606532,-2.736081,17.012004,-2.558912,18.370873,7.04189,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-7.270815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,20.71061,-0.065583,-0.700765,-43.026959,-598.379904,-1068.177947,469.798136,-89.504287,-119.724355,-47.728685,-23.919190,-23.809498,-7.896282,-6.822342,-10.707745,-28.565596,-17.130284,-21.088986,-22.171711,-1.331295,-3.958702,0.254749,-0.671324,-0.001656,50.318354,7.117335,0.114766,-0.098162,0.038356,0.192424,0.044338,-0.028454,-0.044465,-0.301128,-0.554677,-0.036834,-0.130031,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-0.324022,9.293562,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-0.947583,-0.040043,-9.408469,-0.212137,-11.955314,-1.019293,-1.473446,-4.62084,-1069.373833,467.661437,0.30201,-0.203857,-601.712487,44.862861,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,0.011923,-0.060451,-0.265129,18.381836,17.318359,1.426717,-0.468443,-0.417283,-190.670372,15.856777,22.277409,8.174027,13.064012,0.0,-1.276187,-0.020137,-0.042636,11.452984,105.979198,-19.96363,-9.888325,-2.861461,0.282836,-0.007024,0.456731,-14.417671,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,-0.152157,-0.002595,-3.678214,-0.014542,-0.029508,4.816834,-29.009759,0.679915,-33.001492,23.038772,-23.978902,-11.614497,-28.471719,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,-1.257363,-2.793637,-1.932758,-5.008096,30.021879,-23.850720,-16.354808,-35.303704,-0.034569,-0.163184,-109.036398,0.533317,-1.929048,-0.623737,-0.228106,-0.251959,-0.000567,0.566264,-0.000708,-0.02921,-0.104665,-0.001358,0.0,0.039208,0.665644,-0.008999,-11953.712824,-45053.257711,0.377099,-34.616053,-61798.157098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,0.377104,-0.00004,0.379228,-0.012257,-0.107878,9.758669e+08,-46.464078,-4.405604e+08,-1.204418e+08,-11.580668,-1.465191,-33.302382,-225.128986,0.227508,-0.364694,-0.133771,-0.209468,-4.356505,-109.884564,1082.30898,-1.368281,703.889293,-108.409742,491.562669,-105.617978,-17.295406,-943.373846,-598.770792,-25.996269,-22.630448,-220.747724,-14.832889,-0.694428,-12.175933,-0.45614,1.0
1,842,540997,1.0,1541365200,0.0,11897,1545598800,-96.799971,-69.199112,-108.200786,-112.916798,-113.618246,-134.02179,-16.08618,-65.076097,-6.783660,-30.006538,-2.736081,-4.007526,-2.558912,-66.189127,-77.51811,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-11.400815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-1

In [15]:
LEN_TRAIN_FEAT = train_feat.shape[0]
print(f'Наблюдений в датасете после слияния": {LEN_TRAIN_FEAT}')

Наблюдений в датасете после слияния": 860052


После слияния тренировочный датасет имеет больше наблюдей (860052), чем исходный (831653). 

* Это связано с тем, что для некоторых клиентов в данных профиля потребления имеются несколько записей, зафиксированных в разное время. 
* Для таких случаев произошло дублирование некоторых наблюдений из data_train.csv с разными записями признков из features.csv. Дублированными строками считаются те, которые имеют одинаковое значение index_vas.

Избавляться от дубликатов будем с учетом актуальности по временни. 
* Среди дубликатов будем оставлять только те наблдения, у которых разница между временными штампами отклика на услугу ('buy_time_vas') и фиксации записи по признакам профиля потребления ('buy_time_feat') минимальна. 
* Введем новый столбец 'time_delta', который будет отражать указанную разницу во времени.

In [16]:
def buy_time_clear(df):
    df['time_delta'] = abs(df['buy_time_vas'] - df['buy_time_feat'])
    
    # Отсортируем наблюдения в порядке возрастания 'time_delta'
    df.sort_values(['time_delta'], inplace=True, ignore_index=True)
    
    # удалим по порядку все дубликаты, оставляя лишь те, которые встречаются в датафрейме первыми (keep='first')
    df.drop_duplicates(['index_vas'], keep='first', inplace=True)
    
    # Отсортируем датасет по возрастанию index_vas.
    df.sort_values(['index_vas'], inplace=True, ignore_index=True)
    
    return df    

In [17]:
buy_time_clear(train_feat)

,index_vas,id,vas_id,buy_time_vas,target,index_feat,buy_time_feat,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,time_delta
0,0,540968,8.0,1537131600,0.0,3756522,1541970000,-31.559971,327.360888,-45.500786,274.753202,-50.918246,253.64821,49.15382,195.883903,-1.835267,-12.831245,-2.736081,56.284081,-2.558912,-66.189127,-77.51811,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-13.940815,54.495836,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,2.648138,0.785634,0.788392,-0.001884,-0.000023,-0.00003,62.58061,-0.065583,-0.700765,-192.026959,239.787096,-1005.230677,1245.017836,-89.504287,-119.724355,-3.512019,33.247476,-36.759494,-7.896282,-7.997875,-10.697457,-28.551694,-17.130284,-16.088986,-26.171711,-5.331295,1.041298,-3.745251,-1.671324,-0.001656,9.318354,15.117335,0.174766,-0.108162,0.028356,-0.017576,-0.165662,-0.028454,-0.044465,-0.301128,-0.554677,-0.036834,-0.130031,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-3.447583,-0.040043,-9.408469,-0.212137,-11.955314,-1.019293,-1.473446,0.37916,-1006.426563,1242.881137,0.03201,0.196143,236.454513,848.953561,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,0.031923,-0.030451,0.104871,1.381836,5.318359,1.426717,-0.468443,-0.017283,-190.670372,5.856777,7.277409,2.174027,5.064012,0.0,-1.276187,-0.020137,-0.042636,-11.480350,-59.370804,34.03637,-17.888325,0.138539,-0.317164,-0.007024,-0.143269,-14.417671,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,-0.152157,-0.002595,1.321786,-0.014542,-0.009508,-8.183166,35.756904,-14.320085,-40.801492,-5.961228,33.187764,-12.614497,-26.671719,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,-1.257363,-2.793637,-1.932758,-5.008096,-15.978121,-36.800720,-9.354808,-25.320370,-0.034569,-0.163184,-109.036398,0.533317,-1.929048,-0.623737,-0.228106,-0.251959,-0.000567,0.566264,-0.000708,-0.02921,-0.104665,-0.001358,0.0,0.039208,0.665644,-0.008999,-11953.712824,-45175.257711,-0.622901,-30.716053,-61790.157098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,-0.622896,-0.00004,-0.620772,-0.012257,-0.107878,9.698189e+08,-49.224078,-4.405604e+08,-1.204418e+08,-8.634002,-1.465191,-33.302382,810.871014,270.227508,-0.364694,-0.133771,-0.209468,269.643495,-109.884564,364.30898,41.631719,938.889293,-108.409742,-504.437331,-53.617978,-12.295406,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0,4838400
1,1,1454121,4.0,1531688400,0.0,2735973,1531083600,547.270029,238.430888,533.329214,274.803202,527.911754,253.69821,-16.08618,-65.076097,-6.783660,-30.006538,-2.736081,-4.007526,-2.558912,577.880873,566.55189,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0

Проверим нарушилась ли индексация index_vas путем сравнения со сброшенным на предыдущем этапе индеском датасета.

In [18]:
(train_feat.index == train_feat['index_vas']).all()

True

Индексы совпадают, индексация не нарушена.

Сравним количество наблюдений полученного датасета с data_train.csv

In [19]:
print(f'Наблюдений   в   исходном  "data_train.csv": {LEN_TRAIN}')
print(f'Наблюдений в новом тренирвоочном датафрейме: {train_feat.shape[0]}')

Наблюдений   в   исходном  "data_train.csv": 831653
Наблюдений в новом тренирвоочном датафрейме: 831653


Индексация и количество наблюдений после извлечения и добавления признаков потребления в новом тренировочном датасете НЕ НАРУШЕНЫ.

Сохранение нового тренировочного датасета:
* Удалим лишние стобцы с индексами и признаком buy_time из features.csv. 
* Признак time_delta оставим, так как он несет в себе информацию об актуальности признаков профиля потребления на момент отклика клиента на услугу. 
* Сохраним датасет в data_feat_train.pkl.

In [20]:
train_feat.drop(['index_vas','index_feat', 'buy_time_feat'], axis=1).astype('float32').to_pickle('data_feat_train.pkl')

#### Слияние тестовой выборки и набора данных с профилем абонента

In [21]:
test_merge = dd.merge(test_df, features_df, on=['id'], how='inner')
test_feat = test_merge.compute()

In [22]:
buy_time_clear(test_feat)

,index_vas,id,vas_id,buy_time_vas,index_feat,buy_time_feat,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,time_delta
0,0,3130519,2.0,1548018000,1871362,1536526800,-62.899971,-374.279112,-72.600786,-418.406798,-78.018246,-439.51179,17.81382,-31.176097,-6.78366,-30.006538,31.163919,-4.007526,-2.558912,-66.189127,-77.51811,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-9.700815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-92.026959,-2847.354464,-1091.679897,-1755.674564,-89.504287,-119.724355,-88.778686,-35.052524,-53.726160,-7.896282,-7.997875,-10.361439,-28.563689,-17.130284,-22.088986,-48.171711,-25.331295,-4.958702,-8.745251,-1.671324,-0.001656,-24.681646,-3.882665,0.524766,0.421838,-0.021644,0.552424,0.584338,-0.028454,-0.044465,-0.301128,-0.554677,-0.036834,-0.130031,31.116408,-2.60662,28.509788,29.877453,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-3.447583,-0.040043,-9.408469,-0.212137,-11.955314,-1.019293,-2.473446,-1.62084,-1092.875783,-1757.811263,-0.36799,-0.033857,-2850.687047,-2203.639199,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,-0.058077,-0.060451,-0.075129,-5.618164,-10.681641,-1.573283,0.531557,0.582717,-190.670372,-1.143223,-2.722591,-0.825973,-1.935988,0.0,-1.276187,-0.020137,-0.042636,-29.797016,-116.020802,-3.96363,-5.888325,-3.861461,-0.317164,-0.007024,-0.143269,-14.417671,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,-0.152157,-0.002595,-4.678214,-0.014542,-0.079508,-2.183166,-24.826426,-9.320085,-40.651493,-6.961228,-35.112236,5.385503,-11.271719,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,-1.257363,-2.793637,-1.932758,-5.008096,-14.978121,-53.767387,-0.354808,-23.970371,-0.034569,-0.163184,-109.036398,-0.466683,-2.929048,0.376263,-0.228106,-0.251959,-0.000567,-0.433736,-0.000708,-0.02921,-0.104665,-0.001358,0.0,0.039208,-0.334356,-0.008999,-11953.712824,10384.742289,0.377099,-30.716053,-61790.157098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,0.377104,-0.00004,0.379228,-0.012257,-0.107878,-5.726695e+08,-58.544078,-4.405604e+08,-1.204418e+08,6.899332,-1.465191,-32.302382,-252.128986,-39.772492,-0.364694,-0.133771,-0.209468,-35.356505,-109.884564,-876.69102,-5.368281,-247.110707,-108.409742,-512.437331,-84.617978,-17.295406,-970.373846,-613.770792,-25.996269,-37.630448,-258.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,11491200
1,1,2000860,4.0,1548018000,2024591,1532293200,-96.799971,100.290888,-110.740786,140.903202,-116.158246,119.79821,-16.08618,-65.076097,-6.78366,-30.006538,-2.736081,-4.007526,-2.558912,-66.189127,-77.51811,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023

In [23]:
(test_feat.index == test_feat['index_vas']).all()

True

In [24]:
print(f'Наблюдений в исходном  "data_test.csv": {LEN_TEST}')
print(f'Наблюдений в новом тестовом датафрейме: {test_feat.shape[0]}')

Наблюдений в исходном  "data_test.csv": 71231
Наблюдений в новом тестовом датафрейме: 71231


Индексация и количество наблюдений после извлечения и добавления признаков потребления в новом тестовом датасете НЕ НАРУШЕНЫ.

Сохраним датафрейм в data_feat_test.pkl.

In [25]:
test_feat.drop(['index_vas','index_feat'], axis=1).astype('float32').to_pickle('data_feat_test.pkl')